![MIoT_GDPI](img/MIOT_GDPI_header.png)

# Unidad 04 - Introducción a la librería Keras

Una vez finalizado la introducción al paradigma de aprendizaje incremental, nos centraremos ahora en el desarrollo y aplicación de modelos basados en redes neuronales profundas, ya que estos nos permitirán generar sistemas de predicción que tengan en cuenta el tiempo. Para poder desarrollar  este tipo de modelos, emplearemos Tensorflow y el API de Keras3. Dado que la experiencia en este máster estuvo centrada en en `scikit-learn`, emplearemos esta práctica a modo de introducción al uso de Keras con Tensorflow.



La mayor parte del contenido de este Notebook se dedica a explicar el uso básico del API Keras empleando como *backend* Tensorflow. Es crucial que dediquéis tiempo a leer y comprender el material, en lugar de simplemente ejecutar el código. Os invitamos a experimentar modificando y variando el código proporcionado para que podáis explorar las distintas opciones y profundizar en su funcionamiento.



**Importante**: El Notebook contiene un ejercicio sencillo que deberéis desarrollar y enviar por el aula virtual del curso, en la tarea correspondiente


## Referencias útiles para la práctica

1. [Documentación oficial](https://www.tensorflow.org/learn?hl=es-419) de Tensorflow
2. [Guías](https://keras.io/guides/) de Keras



## Tensorflow
TensorFlow es una plataforma de código abierto para machine learning, desarrollada por Google. Permite la computación numérica eficiente, especialmente optimizada para el entrenamiento y despliegue de modelos de aprendizaje automático y redes neuronales profundas a gran escala, soportando su ejecución en una amplia variedad de dispositivos y sistemas, incluyendo CPUs, GPUs y TPUs. Originalmente, Tensorflow requería el desarrollo de los modelos a través de tensores de forma poco amigable, lo que hizo que apareciesen APIs que facilitasen una interfaz para poder emplear esta librería. En particular, Keras fue muy popular hasta el punto que Tensorflow la integró dentro de su desarrollo a partir de la versión 2.

## Keras


Keras es una API de alto nivel escrita en Python que actúa como una interfaz simplificada y amigable para construir y entrenar modelos de aprendizaje profundo. Keras puede  ejecutarse sobre diferentes "motores" (*backends*) como JAX, TensorFlow y PyTorch. Particularmente, la versión Keras 2  está totalmente integrado en TensorFlow 2 a través del módulo de `tf.keras`. En esta práctica usaremos la última versión disponible (Keras 3), lo que requiere instalarla de forma independiente y permite seleccionar el *backend* que usará para generar los modelos.

## 1. Introducción 

**Los procesos industriales secuenciales suelen ser altamente no lineales y presentar importantes dependencias temporales**. Una intervención en una fase temprana sobre un producto intermedio no impacta en el resultado final de forma inmediata, sino después del tiempo que tarda en recorrer el resto de la cadena productiva.

En general los modelos vistos hasta ahora no tienen en cuenta el tiempo. Típicamente son "fotos fijas" de un proceso en el que se recogen las medidas o características de ese momento dado. Por ejemplo, cada observación puede estar compuesta de las medidas de todos los sensores desplegados en la factoría en un segundo concreto.
En un proceso secuencial con dependencias temporales, estaríamos asociando a la variable/s de salida/s, características de entrada que no fueron las que realmente afectaron  y generaron dicha salida.

Típicamente,  para poder trabajar en esta situación en una línea de producción tenemos 2 opciones:
1. Podemos **alinear los datos**, es decir, tratar de crear observaciones que cuadren las entradas concretas que afectaron a la salida escogida. Esto es algo tremendamente complicado si no se conoce perfectamente el proceso productivo, características, tiempos, retrasos, etc.
2. Podemos **trabajar con modelos que recuerden y tengan en cuenta lo que ha pasado en instantes anteriores**.

Trabajar con modelos recurrentes que tengan en cuenta el tiempo es algo que suele beneficiar el desarrollo de predictores en plantas industriales pero son más complicados de desarrollar y entrenar. Librerías típicas como `scikit-learn`no contemplan el  desarrollo de este tipo de modelos pero existen otros frameworks centrados en el desarrollo de redes neuronales profundas en general y que permiten crear redes recurrentes de diferentes tipos. [Tensorflow](https://www.tensorflow.org/?hl=es-419) es uno de los *frameworks* más populares y [Keras](https://keras.io/) es el API más empleada para el desarrollo de modelos con Tensorflow.

In [ ]:

# Importaciones generales
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np


try:
    import seaborn as sns
except ImportError as err:
    !pip install seaborn
    import seaborn as sns



try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

import os

# Asegurarnos de usar Keras 3 con backend TensorFlow
# Es necesario hacerlo antes de cargar Keras
os.environ["KERAS_BACKEND"] = "tensorflow"

# Importaciones de Keras y TensorFlow
try:
    import keras
except ImportError as err:
    !pip install keras
    import keras

try:
    import tensorflow as tf 
except ImportError as err:
    !pip install tensorflow
    import tensorflow as tf 



# Importaciones de Scikit-learn
#Solo para facilitarnos el uso de algunas operaciones típicas
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# Configuraciones para visualización. No es necesario
plt.style.use('seaborn-v0_8-whitegrid') # Estilo de gráficos
sns.set_palette('viridis') # Paleta de colores


# Verificación de versiones
#Existen diferentes versiones y compatibilidades. Es importantes saber lo que estamos ejecutando
#Para estos ejemplos queremos ejecutar el API de Keras3 (standalone) y tensorflo3>2.16
print(f"Versión de Keras: {keras.__version__}")
print(f"Backend de Keras: {keras.backend.backend()}")
print(f"Versión de TensorFlow: {tf.__version__}")



# Verificar si TensorFlow puede usar GPU en nuestro sistema (opcional, pero bueno saberlo)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"GPU disponible: {gpu_devices}")
else:
    print("GPU no encontrada, se usará CPU.")

## 2. Dataset

Para el desarrollo del ejemplo emplearemos el *dataset* público de Kaggle: [Concrete Compressive Strength Data Set](https://www.kaggle.com/datasets/elikplim/concrete-compressive-strength-data-set/data).
El repositorio contiene una descripción del conjunto de datos y de sus características:


**Concrete Compressive Strength Data Set**


*Abstract: Concrete is the most important material in civil engineering. The
concrete compressive strength is a highly nonlinear function of age and
ingredients. These ingredients include cement, blast furnace slag, fly ash,
water, superplasticizer, coarse aggregate, and fine aggregate.*

| **Nombre** | **Tipo de dato** | **Medida** | **Descripción** |
|---|---|---|---|
| Cement (component 1) | quantitative | kg in a m³ mixture | Input Variable |
| Blast Furnace Slag (component 2) | quantitative | kg in a m³ mixture | Input Variable |
| Fly Ash (component 3) | quantitative | kg in a m³ mixture | Input Variable |
| Water (component 4) | quantitative | kg in a m³ mixture | Input Variable |
| Superplasticizer (component 5) | quantitative | kg in a m³ mixture | Input Variable |
| Coarse Aggregate (component 6) | quantitative | kg in a m³ mixture | Input Variable |
| Fine Aggregate (component 7) | quantitative | kg in a m³ mixture | Input Variable |
| Age | quantitative | Day (1\~365) | Input Variable |
| **Concrete compressive strength** | quantitative | MPa | **Output Variable** |


Para facilitar el acceso a los datos, descargamos la última versión del dataset empleando `kagglehub`.

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("elikplim/concrete-compressive-strength-data-set")

print("Path to dataset files:", path)
#Nombre del fichero descargado: concrete_data.csv
df_concrete=pd.read_csv(path+"/concrete_data.csv")



## 3. Análisis de los datos


**IMPORTANTE**: *el objetivo  de este Notebook es desarrollar modelos con Keras y Tensorflow, por lo que simplificaremos al máximo las fases de análisis y preprocesado. El rendimiento del modelo no es prioritario. Conocéis de materias previas la **importancia de analizar y preparar los datos  para mejorar la precisión de los modelos**.*



In [ ]:
print(f"Numero de filas: {len(df_concrete)}")
print(f"Numero de características: {len(df_concrete.columns)}")
print("#"*50)
print("\nInformación general del dataset:")
df_concrete.info()

print("#"*50)

 # Vistazo rápido a los datos
print("\nPrimeras filas del dataset:")
print(df_concrete.head())
print("#"*50)


# Comprobar valores nulos (aunque ya se ven en el info) 
print("\nValores nulos por columna:")
print(df_concrete.isnull().sum())

Aunque el análisis detallado de los datos no es el objetivo de este Notebook, **dividiremos**, ya en este punto, **el dataset para evitar el  llamado "sesgo de espionaje"** y así poder estudiar el conjunto y con ello generar operaciones de preprocesado que permitan mejorar el futuro modelo.

In [ ]:
# Establecemos una semilla para la reproducibilidad. 
#Emplearemos esta semilla para todos los procesos pseudoaleatorios
SEED=42



#Dividimos en entrenamiento, validación y test 
# Usaremos un 70% para entrenamiento, un 15% para Validación y un 15% para Test.
#Nos devuelve un array de numpy

concrete_train, concrete_pruebas_aux=train_test_split(df_concrete, test_size=0.30, random_state=SEED)#%70% para entrenamiento
concrete_val, concrete_test=train_test_split(concrete_pruebas_aux, test_size=0.50, random_state=SEED)#15% Val y 15% Test



#Convertimos a DataFrames para mejor visualización (opcional)
df_concrete_train = pd.DataFrame(concrete_train, columns=df_concrete.columns)
df_concrete_val = pd.DataFrame(concrete_val, columns=df_concrete.columns)
df_concrete_test = pd.DataFrame(concrete_test, columns=df_concrete.columns)

print(f"Datos de entrenamiento: ({len(df_concrete_train)},{len(df_concrete_train.columns)})")
print(f"Datos de validacion: ({len(df_concrete_val)},{len(df_concrete_val.columns)})")
print(f"Datos de test: ({len(df_concrete_test)},{len(df_concrete_test.columns)})")





#Analizamos solo el conjunto de entrenamieto para evitar el sesgo de espionaje
print("Estadísticas descriptivas del conjunto de entrenamiento:")
print(df_concrete_train.describe().transpose())


for col in df_concrete_train.columns:
    sns.histplot(data=df_concrete_train[col], bins=50)
    plt.show() # Seaborn configura implícitamente el objeto plt


plt.figure(figsize=(16, 6))
sns.heatmap(df_concrete_train.corr(),vmin=-1, vmax=1, annot=True, cmap='BrBG')#colores más oscuros marcan más correlación
plt.title('Correlation Heatmap');



## 4. Preparación de los datos
tal y como se mencionó anteriormente, realizaremos una preparación mínima de los datos cara al desarrallo del modelo. En particular estandarizaremos los valores de las características. Sería conveniente realizar otro tipo de operaciones cara a continuar mejorando el modelo.

### 4.1 Estandarización de los datos con Keras


En lugar de estandarizar los datos a través de una operación de `scikit-learn` (sistema que ya conocéis), lo haremos desde el propio Keras generando una [capa de preprocesado](https://keras.io/api/layers/preprocessing_layers/). Esto tiene como ventaja poder incluirlo en el propio modelo y que este sea autosuficiente. Debéis de recordar que el entrenamiento de los modelos es solo la primera fase, luego es necesario desplegarlos en producción y trabajar con ellos. Poder guardar con el modelo toda la información necesaria para poder emplearlo (ej. capa de estandarización), simplifica su empleo.

In [ ]:
#separamos las entradas de las salidas para los 3 conjuntos
def separar_inputs_outputs(dataset):
    if 'concrete_compressive_strength' in dataset.columns: 
        return dataset.drop('concrete_compressive_strength', axis=1), dataset['concrete_compressive_strength']
    else: return None, None


df_concrete_train_X, df_concrete_train_y=separar_inputs_outputs(df_concrete_train)
df_concrete_val_X, df_concrete_val_y=separar_inputs_outputs(df_concrete_val)
df_concrete_test_X, df_concrete_test_y=separar_inputs_outputs(df_concrete_test)





#creamos una capa de preprocesado para normalizar
normalization_layer = keras.layers.Normalization(axis=-1) # Normaliza a través del eje de características

#adapt es el método que calcula los datos de estandarización pero
# Keras espera un array NumPy o Tensor, no un DataFrame directamente.
#df.values genera un array de numpy
normalization_layer.adapt(df_concrete_train_X.values)
print(f"  Medias aprendidas: {normalization_layer.mean.numpy()}")
print(f"  Varianzas aprendidas: {normalization_layer.variance.numpy()}")

#Probamos la estandarización con los datos de entrenamiento
df_concrete_train_X_scaled=normalization_layer(df_concrete_train_X.values)
print("Media de las entradas del dataset de entrenamiento escalado (debería ser cercana a 0):")
#La capa devuelve un tensor que es necesario pasar a un array de numpy
print(df_concrete_train_X_scaled.numpy().mean())
print("Desviación estándar de las entradas del dataset de entrenamiento escalado (debería ser cercana a 1):")
print(df_concrete_train_X_scaled.numpy().std())





## 5. Desarrollo  del modelo con Keras y Tensorflow
### 5.1 Definición del modelo

Keras permite general modelos empleando dos opciones:
1. El [API secuencial](https://keras.io/guides/sequential_model/): Permite generar redes densas con diferentes capas donde cada capa tiene exactamente un tensor de entrada y un tensor de salida. Se generan capas que se unen (apilan) secuencialmente.
2. El [API funcional](https://keras.io/guides/functional_api/): permite crear modelos más flexibles que los secuenciales. Esta API permite gestionar modelos con topologías no lineales. Permite crear "grafos de capas".



Vamos a definir nuestra red neuronal usando el API Secuencial de Keras (`keras.Sequential`), ya que es suficiente crear un modelo que "apile"  capas una tras otra y cubre nuestras necesidades.


Definiremos una red simple:
- **Capa de Entrada**: Definida de forma explícita a través de  `keras.Input`. Su parámetro `shape` debe coincidir con el número de características de nuestro dataset (8 en nuestro caso). Es importante destacar que no es necesario crear de forma explícita esta capa, se puede obtener la información de forma indirecta en el  proceso de entrenamiento pero definirla de forma explícita nos permite ver un resumen completo de la red generada (incluyendo los pesos), en el momento que se define.
- **Capas Ocultas**: Usaremos un par de capas densas (`keras.layers.Dense`) con activación ReLU (`relu`). ReLU es una opción común y eficiente para capas ocultas en redes profundas. El número de neuronas es un hiperparámetro a optimizar (empezaremos con 64 y 32).
- **Capa de Salida**: Una única neurona (Dense(1)) ya que es un problema de regresión (predecir un solo valor continuo). Usaremos la función de activación `linear`), ya que  queremos que la salida pueda tomar cualquier valor real, no limitado a un rango específico.

Explicación del código:
- keras.Sequential([...]): Crea un modelo donde las capas se ejecutan en secuencia.
- keras.Input(shape=(n_features,)): Define formalmente la forma de los datos de entrada. Ayuda a Keras a construir el modelo correctamente.
- keras.layers.Dense(units, activation=...): Define una capa totalmente conectada (densa). `units` es el número de neuronas. `activation` es la función de activación para cada una de las neuronas de esa capa.
- model.summary(): Muestra las capas, la forma de salida de cada capa y el número de parámetros entrenables. Es útil para verificar la arquitectura definida.
    
   

In [ ]:
# Establece la semilla para los números aleatorios con  keras.utils.set_random_seed. Esto establecerá:
# 1) `numpy` seed
# 2) backend random seed
# 3) `python` random seed
#Permitir la reproducibilidad es clave para poder comparar
keras.utils.set_random_seed(SEED)


# Obtenemos el número de características para la capa de entrada
n_features = len(df_concrete_train_X.columns)
print(f"Número de características de entrada: {n_features}")

# Construcción del modelo secuencial
model = keras.Sequential(
    [
        keras.Input(shape=(n_features,), name="input_layer"), # Capa de entrada explícita. 
                                                              #No es obligatoria pero nos permite ver el resumen 
                                                              #completo de la red al crearla, incluyendo los pesos
        normalization_layer, #capa de normalización que generamos anteriormente. El modelo es autosuficiente
        keras.layers.Dense(64, activation='tanh', name='hidden_layer_1'), # 1ra capa oculta con 32 neuronas y activación ReLU
        keras.layers.Dense(32, activation='tanh', name='hidden_layer_2'), # 2da capa oculta con 16 neuronas y activación ReLU
        keras.layers.Dense(1, activation='linear', name='output_layer')   # Capa de salida con 1 neurona (regresión) y activación lineal
    ],
    name="Concrete_Strength_Predictor" # Nombre opcional para el modelo
)


# Mostrar un resumen de la arquitectura del modelo
print("\nResumen del modelo:")
model.summary()

Keras permite añadir capas a un modelo con el método `add`, lo que puede ser útil para implementar funciones que nos permitan **generar modelos dinámicamente**, lo que podremos usar para la optimización de hiperparámetros.La siguiente función os muestra un ejemplo:

In [ ]:
def crear_modelo(input_neurons, hidden_layer_neurons, preprocesing_layers, name_model):
    #input layer
    model = keras.Sequential( [keras.Input(shape=(input_neurons,), name="input_layer")], name=name_model)
   
 
    #preprocesing layers
    for layer in preprocesing_layers:
       model.add(layer)
        
    #Hidden layers
    for i, neurons in   enumerate(hidden_layer_neurons,start=1):
       model.add(keras.layers.Dense(neurons, activation='relu', name=f'hidden_layer_{i}'))

    #output layer
    model.add(keras.layers.Dense(1, activation='linear', name='output_layer'))
    
    return model 
       


modelo2=crear_modelo(input_neurons=8, hidden_layer_neurons=[64,32,16], preprocesing_layers=[normalization_layer], name_model="Concrete_Strength_Predictor2")
print("\nResumen del modelo2:")
modelo2.summary()
    

### 5.2 Compilación del modelo


Antes de entrenar, necesitamos "compilar" el modelo para configurar su proceso de entrenamiento. Esto implica definir:

* **Optimizador**: Algoritmo que ajusta los pesos de la red durante el entrenamiento (ej. Adam, SGD, RMSprop). Adam es una opción robusta y popular. Tenéis [aquí](https://keras.io/api/optimizers/) un listado de los disponibles. La configuración de los parámetros del optimizador sería algo para optimizar en la búsqueda hiperparamétrica.

* **Función de Pérdida** (*Loss Function*): Mide qué tan bien se desempeña el modelo durante el entrenamiento. Para regresión, mean_squared_error (MSE) es una opción muy típica. Tenéis [otras](https://keras.io/api/losses/) disponibles.


* **Métricas**: Funciones para evaluar el rendimiento del modelo (ej. mean_absolute_error - MAE). **Las métricas no se usan para optimizar el modelo, solo para monitorizarlo**. Tenéis [aquí](https://keras.io/api/metrics/) un listado de las disponibles

In [ ]:
model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001), # Optimizador Adam con tasa de aprendizaje típica
        loss='mean_squared_error',          # Función de pérdida para regresión (MSE)
        metrics=[keras.metrics.MeanAbsoluteError(name='mae')] # Métrica para monitorear (MAE)
    )

print("Modelo compilado exitosamente.")
print(f"Optimizador: {type(model.optimizer).__name__}")
print(f"Función de pérdida: {model.loss}")
print("Métricas: ")
#metricas incluye la empleada como función de pérdida
for m in model.metrics:
    print(m.name)


### 5.3  Entrenamiento del modelo

Ya tenemos definido y configurado el modelo para poder entrenarse con nuestro  dataset de entrada. Recordad que el **modelo contiene una capa de normalización**, por lo que **los datos que se le pasen para entrenar NO pueden estar normalizados**(lo hará el propio modelo).


El método `fit` es el empleado para entrenar. Tenéis [aquí](https://keras.io/api/models/model_training_apis/#fit-method) disponible todos los parámetros posibles. En este caso emplearemos:

* **epochs**: Número de veces que el modelo verá el conjunto de datos completo.
* **batch_size**: Número de muestras procesadas antes de actualizar los pesos (entrenamiento con *mini-batches*).
* **validation_data**: conjunto de datos sobre los que validaremos el entrenamiento del modelo. Lo podemos emplear para comparar configuraciones o para parar el entrenamiento de forma prematura (*early_stopping*), lo que puede ayudar a detectar el *overfitting*.

El método `fit` devuelve un objeto *History* que contiene información sobre el proceso de entrenamiento (pérdida y métricas en cada época) que podremos utilizar posteriormente para graficar el proceso de entreno y poder valorar si tenemos *overfitting* o *underfitting*.


**IMPORTANTE**: cada vez que ejecutéis el entrenamiento, el modelo parte de su último estado. Si queréis volver a entrenarlo desde cero, entonces tenéis que volver a crearlo.


In [ ]:
##IMPORTANTE: si se vuelve a ejecutar esta celda sin generar el modelo desde 0
##Partiréis de los pasos del último entrenamiento
# Definimos el número de épocas y el tamaño del batch
EPOCHS = 1000
BATCH_SIZE = 256

# Entrenamos el modelo
history = model.fit(
    df_concrete_train_X.values,
    df_concrete_train_y.values,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(df_concrete_val_X.values, df_concrete_val_y.values),
    verbose=1 # Muestra una barra de progreso por época (0=silencioso, 1=barra, 2=una línea por época)
)

### 5.4 Evaluación del Modelo

Una vez entrenado el modelo, tenemos que evaluar su rendimento. Lo haremos desde dos puntos de vista:

1. **Visualizando el historial de entrenamiento**: Graficaremos la pérdida y la/s métrica/s disponibles (el MAE en este caso) tanto para el conjunto de entrenamiento como para el de validación a lo largo de las épocas. Esto nos ayuda a diagnosticar problemas como el *overfitting* (cuando la pérdida de validación empieza a aumentar mientras la de entrenamiento sigue bajando) o el *underfitting* (cuando ambas pérdidas son muy altas).
2. **Evaluando en el conjunto de Test**: Usaremos el método `evaluate` con los datos de Test, que el modelo nunca ha visto, para generar la  estimación final e imparcial del rendimiento del modelo.


In [ ]:
def graficas_entrenamiento(hist):
    hist=pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(hist['epoch'], hist['loss'], label='Pérdida Entrenamiento')
    plt.plot(hist['epoch'], hist['val_loss'], label = 'Pérdida Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida (MSE)')
    plt.title('Pérdida (MSE) durante Entrenamiento')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    # Asegúrate de usar el nombre correcto de la métrica (puede variar ligeramente)
    mae_key = 'mae' if 'mae' in hist.columns else list(hist.columns)[1] # Intenta encontrar la clave MAE
    val_mae_key = 'val_' + mae_key
    plt.plot(hist['epoch'], hist[mae_key], label='MAE Entrenamiento')
    plt.plot(hist['epoch'], hist[val_mae_key], label = 'MAE Validación')
    plt.xlabel('Épocas')
    plt.ylabel('Error Absoluto Medio (MAE)')
    plt.title('MAE durante Entrenamiento')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()


#1. Graficar el entrenamiento por épocas. Ayuda a valorar el posible (sobre/infra)entrenamiento
graficas_entrenamiento(history)

# 2. Evaluar en el conjunto de prueba
print("Evaluando el modelo en el conjunto de prueba. Métrica general")
test_loss, test_mae = model.evaluate(df_concrete_test_X.values, df_concrete_test_y.values, verbose=0)

print(f"Pérdida en el conjunto de Test (MSE): {test_loss:.4f}")
print(f"Error Absoluto Medio (MAE) en el conjunto de Test: {test_mae:.4f}")

y_pred_test = model.predict(df_concrete_test_X.values) # Aplanar para que tenga la misma forma que y_test

# Calcular R^2 score (opcional, pero útil en regresión)
r2 = r2_score(df_concrete_test_y.values, y_pred_test)
print(f"Coeficiente de Determinación (R²) en el conjunto de Test: {r2:.4f}")

#Scatterplot entre los valores reales y los predichos
plt.figure(figsize=(12, 5))
sns.regplot(x=df_concrete_test_y.values, y=y_pred_test.flatten())
plt.title("Concrete_Strength_Predictor")
plt.xlabel('Valores reales')
plt.ylabel('Predicciones')


### 5.5 Almacenamiento del modelo
Una vez entrenado el modelo es necesario guardarlo para poder desplegarlo en producción en el futuro. Este es un proceso sencillo en Keras y que se realiza a través del método `save` que podéis consultar [aquí](https://keras.io/api/models/model_saving_apis/model_saving_and_loading/#save-method). Este método guarda toda la información necesaria para el modelo el un archivo comprimido `.keras` con el siguiente contenido:

- La arquitectura del modelo.
- Los pesos del modelo.
- El estado del optimizador (si es necesario).
  


In [ ]:
PATH_SAVED_MODEL="./concrete_model.keras"

model.save(filepath=PATH_SAVED_MODEL)
del model #eliminamos de memoria el modelo para estar seguros de que no lo tenemos cargado



### 5.6 Cargar el modelo
Una vez tenemos un modelo entrenado y almacenado, deberemos desplegarlo en producción. Para ello cargaremos el modelo a través del método `load_model`

In [ ]:
model=keras.saving.load_model(PATH_SAVED_MODEL)
model.summary()

Una vez tenemos cargado el modelo nuevamente, podemos emplearlo para predecir en producción pero también podríamos usarlo para recuperar los pesos y realizar un reentrenamiento para minimizar un posible *concept drift*.


In [ ]:
##Podemos acceder a los pesos de la red (capa a capa)
model.layers[1].get_weights()[0] #pesos de la capa oculta

### 5.7 Predecir con el modelo en producción
Una vez cargado el modelo podemos emplearlo para generar predicciones de los datos entrantes en producción. Simularemos con flujo de datos con el conjunto de Test para ver cómo se realizaría:

In [ ]:
#Suponiendo que obtenemos datos desde un flujo de datos en tiempo real
#en este caso no tenemos acceso a la salida (output) 

for observacion in range(5):
    fila=df_concrete_test_X.iloc[[observacion]].values
    prediction=model.predict(fila, verbose=0)
    print(f"La predicción es {prediction}")

    





#### EJERCICIO PARA DESARROLLAR Y ENTREGAR EN EL AULA VIRTUAL

Con el objetivo de aprender como realizar ciertas operaciones con la interfaz de Keras, generaréis y probaréis un nuevo modelo con los siguientes cambios:
* **Modificaréis la función de activación** de las capas ocultas empleando la **tangente hiperbólica**
* **Añadiréis regularización** al modelo para evitar el sobreentrenamiento empleando  **DropOut** en cada una de las capas ocultas.
    - Un `rate' del 20% es un buen punto de partida.
    - La capa de entrada no tendrá el **DropOut**.

* Añadiréis la **métrica de Root Mean Squared Error** (junto al MAE).     
  
* Estudiaréis el concepto de ***callbacks*** en Keras y **añadiréis un *EarlyStop*** a vuestro modelo.
    - Monitoriza la pérdida de validación.
    - Esperarés 6 rondas para terminar el entrenamiento.
* Evaluaréis el modelo y visualizaréis las métricas de entrenamiento

Podéis escoger los valores que consideréis oportunos para los parámetros para los que no se especifica un cambio (ej. capas ocultas y neuronas)

**IMPORTANTE**: el modelo se generará dinámicamente a través de una función `crear_modelo_ej` (como en el ejemplo previo). Deberéis modificar la función de forma oportuna.

Este ejercicio contendrá los diferentes pasos en el desarrollo del modelo:
- Definición del modelo.
- Compilación del modelo.
- Entrenamiento del modelo.
- Evaluación del Modelo.


